In [51]:
import os
import pandas as pd
import panel as pn
import altair as alt
import scipy.stats as stats
from toolz.curried import pipe

def json_dir(data, data_dir='altairdata'):
    os.makedirs(data_dir, exist_ok=True)
    return pipe(data, alt.to_json(filename=data_dir + '/{prefix}-{hash}.{extension}') )


alt.renderers.enable('default')
pn.extension('vega')

alt.data_transformers.register('json_dir', json_dir)
alt.data_transformers.enable('json_dir', data_dir='mydata')



DataTransformerRegistry.enable('json_dir')

In [15]:
# Lendo os dados do arquivo XLSX
file_path = '../../scrapper-dataset/excel/all-companies.xlsx'

data = pd.read_excel(file_path)
data['Data_only'] = data['data_criacao'].dt.date;


In [65]:
Q1 = data['sla'].quantile(0.25)
Q3 = data['sla'].quantile(0.75)
IQR = Q3 - Q1

limite_superior = Q3 + 1.5 * IQR

df_sem_outliers = data[(data['sla'] <= limite_superior)]

In [66]:
df_sem_outliers.sort_values('sla', ascending=False).head()


,id,titulo,status,data_criacao,reclamacao,estado,cidade,categoria,problema,produto,voltaria_fazer_negocio,nota,respondido_em,sla,empresa,Data_only
105978,158657355,Entrega do pedido,Respondida,2023-02-03 09:12:00,Entrei no site ver o status do meu pedido e es...,SP,Paulínia,Não encontrei meu problema,Atraso na entrega,Problemas Gerais,False,NaN,2023-02-25 16:18:00,1926360.0,amazon,2023-02-03
127059,134000853,Troca de TV por defeito não realizada,Respondida,2021-12-15 10:16:00,No dia 17/11 comprei uma TV 65 LG 4k LED 65UP7...,SP,Osasco,Eletroeletrônicos,Tela manchada,TV,False,NaN,2022-01-06 17:21:00,1926300.0,casas-bahia-loja-online,2021-12-15
80204,165199855,O meu pedido sai para entrega todos os dias e ...,Respondida,2023-05-24 08:06:00,"O site atualiza que o pedido sai para emtrega,...",MG,Pouso Alegre,Logística e Entrega Rápida,Atraso na entrega,Produtos,False,NaN,2023-06-15 15:07:00,1926060.0,amazon,2023-05-24
9253,161440733,potabilidade da tim fibra,Respondida,2023-03-27 11:29:00,"começo da semana passada , referente a semana ...",SP,São Paulo,Celulares e Smartphones,Qualidade do serviço prestado,Acessórios e Periféricos,False,NaN,2023-04-18 18:26:00,1925820.0,live-tim,2023-03-27
31553,166027715,Ligações Abusivas,Respondida,2023-06-07 12:45:00,Tim me liga de 15 em 15 minutos fazendo cobran...,PR,Campina da Lagoa,Telefonia celular,Mau Atendimento,Planos pós-pagos,False,NaN,2023-06-29 19:40:00,1925700.0,tim-celular,2023-06-07


In [68]:
df_media_resolucao_cat = df_sem_outliers.groupby('categoria')['sla'].mean().reset_index()
df_media_resolucao_cat = df_media_resolucao_cat.sort_values('sla', ascending=False)
df_top_10_cat = df_media_resolucao_cat.head(10)
df_top_10_cat['sla'] = df_top_10_cat['sla'] / 60 / 60 / 24 # Converter para dias

df_media_resolucao_prob = df_sem_outliers.groupby('problema')['sla'].mean().reset_index()
df_media_resolucao_prob = df_media_resolucao_prob.sort_values('sla', ascending=False)
df_top_10_prob = df_media_resolucao_prob.head(10)
df_top_10_prob['sla'] = df_top_10_prob['sla'] / 60 / 60 / 24 # Converter para dias



df_media_resolucao_prod = df_sem_outliers.groupby('produto')['sla'].mean().reset_index()
df_media_resolucao_prod = df_media_resolucao_prod.sort_values('sla', ascending=False)
df_top_10_prod = df_media_resolucao_prod.head(10)
df_top_10_prod['sla'] = df_top_10_prod['sla'] / 60 / 60 / 24 # Converter para dias


C:\Users\Thiag\AppData\Local\Temp\ipykernel_21732\2401783274.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_top_10_cat['sla'] = df_top_10_cat['sla'] / 60 / 60 / 24 # Converter para dias
C:\Users\Thiag\AppData\Local\Temp\ipykernel_21732\2401783274.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_top_10_prob['sla'] = df_top_10_prob['sla'] / 60 / 60 / 24 # Converter para dias
C:\Users\Thiag\AppData\Local\Temp\ipykernel_21732\2401783274.py:16: SettingWithCopyWarning: 
A value is trying to be set o

In [44]:
lineData = data[['Data_only','status']].dropna()


In [45]:
chart = alt.Chart(lineData).mark_line().encode(
    x=alt.X('Data_only:T', title='Data'),
    y=alt.Y('count():Q',  title='Quantidade de Reclamações'),
    color=alt.Color('status:N', legend=alt.Legend(title='status')),
    tooltip=['data_criacao:T', 'status:N', 'count()']
).properties(
    width=1500,
    height=400,
    title='Quantidade de Reclamações ao Longo do Tempo por Status',
).interactive()


chart


alt.Chart(...)

In [69]:
chart_cat = alt.Chart(df_top_10_cat).mark_bar().encode(
    x=alt.X('categoria:N', title='categoria', sort=None),
    y=alt.Y('sla:Q', title='Tempo Médio de Resolução (dias)'),
    color=alt.Color('categoria:N', legend=None),
    tooltip=['categoria:N', alt.Tooltip('sla:Q', format='.2f')]
).properties(
    width=600,
    title='Tempo Médio de Resolução por Categoria'
)

chart_prob = alt.Chart(df_top_10_prob).mark_bar().encode(
    x=alt.X('problema:N', title='problema', sort=None),
    y=alt.Y('sla:Q', title='Tempo Médio de Resolução (dias)'),
    color=alt.Color('problema:N', legend=None),
    tooltip=['problema:N', alt.Tooltip('sla:Q', format='.2f')]
).properties(
    width=600,
    title='Tempo Médio de Resolução por Problema'
)


chart_prod = alt.Chart(df_top_10_prod).mark_bar().encode(
    x=alt.X('produto:N', title='produto', sort=None),
    y=alt.Y('sla:Q', title='Tempo Médio de Resolução (dias)'),
    color=alt.Color('produto:N', legend=None),
    tooltip=['produto:N', alt.Tooltip('sla:Q', format='.2f')]
).properties(
    width=600,
    title='Tempo Médio de Resolução por Produto'
)
chart_cat | chart_prob | chart_prod

alt.HConcatChart(...)